<a href="https://colab.research.google.com/github/NathanArsement/jstest/blob/main/VitTimetable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install icalendar

In [ ]:
import icalendar
import datetime
from dateutil import rrule as dateutil_rrule
import re


days = ("MO", "TU", "WE", "TH", "FR", "SA", "SU")


RECORD_SIZE = 16
SHIFTS = {
    "REGISTRATION": 6,
    "CLASS_NUMBER": 7,
}


def transform_arrear_course(data: list[str], line_index: int, lines_removed: set[int]) -> int:
    if data[line_index + SHIFTS["REGISTRATION"]].strip() != "Reregistered":
        return 0
    lines_removed.add(line_index + SHIFTS['CLASS_NUMBER'])
    lines_removed.add(line_index + RECORD_SIZE)
    return 2


def transform_arrear_courses(data: list[str], start_index: int) -> list[str]:
    lines_removed = set()
    line_index = start_index
    while line_index < len(data):
        iterator_shift = transform_arrear_course(data, line_index, lines_removed)
        line_index += RECORD_SIZE + iterator_shift
    return [x for i, x in enumerate(data) if i not in lines_removed]

def get_courses(text: str) -> dict[str, dict]:
    """
    Converts the text copied from the course list in, VTopCC >> Academics >> Time Table, to a list of courses with the
    relevant data.
    """
    data = text.splitlines()
    data = [x for x in data if x.strip()]
    # print('\n'.join(data))
    start_index = data.index('1')
    data = transform_arrear_courses(data, start_index)
    courses = {}
    for line_index in range(start_index, len(data), RECORD_SIZE):
        slot = data[line_index + 7]
        header = (data[line_index + 2].split(' - '))
        course_code = header[0]
        courses[course_code] = {
            'title': header[1],
            'LTPJC': tuple(map(float, data[line_index + 4].split())),
            'class_code': data[line_index + 7],
            'slot': slot,
            'venue': data[line_index + 9],
            'professor': data[line_index + 10].rstrip(' -')
        }
    return courses


def get_slot_times(start_times: list[str], end_times: list[str]) -> list[(datetime.time, datetime.time)]:
    """
    Slots times from first two lines of timetable text becomes,
     list(tuple(start_time, end_time), ...) where,
     time = list(hours, minutes)
    """
    for times in (start_times, end_times):
        noon_flag = False
        # Bug - [12am, 1am) would be evaluated as pm
        previous_time = datetime.time(0)
        for index, time in enumerate(times):
            if time == "Lunch":
                continue
            if time == "-":
              continue
            time_list = list(int(component) for component in time.split(":"))
            time = datetime.time(*time_list)
            if time < previous_time:
                noon_flag = True
            if noon_flag:
                time_list[0] += 12
                time = datetime.time(*time_list)
            if not noon_flag:
                previous_time = time
            times[index] = time
    slot_times = list(zip(start_times, end_times))
    return slot_times


# def add_events(
#         day_rows: list[str],
#         slot_timings: list[tuple[datetime.time]],
#         courses: dict[str, dict],
#         semester_dates: list[datetime.date],
#         calendar: icalendar.cal.Calendar
# ) -> None:
#     """Goes through the list of slots in the days and adds any classes found to the calendar as events"""
#     for day_index, day_row in enumerate(day_rows):
#         for slot_index, slot_cell in enumerate(day_row):
#             if "-" not in slot_cell or slot_cell == "-":
#                 continue
#             slot_cell = slot_cell.split("-")
#             slot_course = slot_cell[1]
#             slot_venue = "-".join(slot_cell[3:5])
#             course = courses[slot_course]
#             event = icalendar.Event()
#             event['summary'] = course['title']
#             event['location'] = slot_venue
#             newline_character = "\n"
#             event['description'] = f"""course_code = {slot_course}
# {newline_character.join((" = ".join(map(str, item)) for item in course.items()))}"""
#             semester_start = semester_dates[0]
#             start_time, end_time = slot_timings[slot_index]
#             ical_time_format = '%Y%m%dT%H%M%S'
#             dtstart = datetime.datetime.combine(semester_start, start_time)
#             event['dtstart'] = dtstart.strftime(ical_time_format)
#             event['dtend'] = datetime.datetime.combine(
#                 semester_start,
#                 end_time
#             ).strftime(ical_time_format)
#             event['dtstamp'] = datetime.datetime.now().strftime(ical_time_format)
#             event['tzinfo'] = "Asia/Kolkata"
#             event['uid'] = str(day_index)+"-"+str(slot_index)
#             print(days[day_index])
#             event['rrule'] = icalendar.vRecur(freq='WEEKLY', byday=days[day_index])
#             exdates = []
#             for start_date, end_date in zip(semester_dates[1::2], semester_dates[2::2]):
#                 exdates.extend(dateutil_rrule.rrule(
#                     dateutil_rrule.WEEKLY,
#                     dtstart=start_date,
#                     until=end_date,
#                     byweekday=days[day_index]
#                 ))
#             event['exdate'] = [date.strftime("%Y%m%d") for date in exdates]
#             calendar.add_component(event)


def add_events(
    day_rows: list[str],
    slot_timings: list[tuple[datetime.time, datetime.time]],
    courses: dict[str, dict],
    semester_dates: list[datetime.date],
    calendar: icalendar.cal.Calendar
) -> None:
    """Add class events to the calendar from timetable slots."""

    # Map day index to RRULE days
    days = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']

    for day_index, day_row in enumerate(day_rows):
        for slot_index, slot_cell in enumerate(day_row):
            if "-" not in slot_cell or slot_cell.strip() == "-":
                continue

            slot_cell_parts = slot_cell.split("-")
            if len(slot_cell_parts) < 4:
                continue  # Skip badly formatted cells

            slot_course = slot_cell_parts[1].strip()
            slot_venue = "-".join(slot_cell_parts[3:5]).strip()

            if slot_course not in courses:
                continue  # Skip unknown courses

            course = courses[slot_course]

            event = icalendar.Event()
            event.add('summary', course['title'])
            event.add('location', slot_venue)

            event.add('description', f"course_code = {slot_course}\n" +
                      "\n".join(f"{k} = {v}" for k, v in course.items()))

            semester_start = semester_dates[0]
            start_time, end_time = slot_timings[slot_index]

            dtstart = datetime.datetime.combine(semester_start, start_time)
            dtend = datetime.datetime.combine(semester_start, end_time)

            event.add('dtstart', dtstart)
            event.add('dtend', dtend)
            event.add('dtstamp', datetime.datetime.now())
            event.add('uid', f"{day_index}-{slot_index}-{datetime.datetime.now().timestamp()}")

            # Add RRULE for weekly repeat
            event.add('rrule', {'freq': 'weekly', 'byday': days[day_index]})

            # Add EXDATEs for semester breaks
            for i in range(1, len(semester_dates) - 1, 2):
                break_start = semester_dates[i]
                break_end = semester_dates[i + 1]

                exdates = list(dateutil_rrule.rrule(
                    dateutil_rrule.WEEKLY,
                    dtstart=datetime.datetime.combine(break_start, start_time),
                    until=datetime.datetime.combine(break_end, start_time),
                    byweekday=day_index
                ))

                for exdate in exdates:
                    event.add('exdate', exdate)

            calendar.add_component(event)



def split_timetable_line(line):
    return line.split('\t')


def split_text(page_text: str) -> tuple[str, str]:
    """
    Splits the text copied from VTopCC >> Academics >> Time Table to,
    courses_text, timetable_text
    """
    split_text = page_text.split("Total Number Of Credits:")
    courses_text = split_text[0]
    timetable_text = split_text[1][split_text[1].index("THEORY"):]
    return courses_text, timetable_text


def generate_calendar(
        page_text : str,
        semester_dates: list[datetime.date]
) -> bytes:
    """
    semester_dates: End date is exclusive
    """
    courses_text, timetable_text = split_text(page_text)
    courses = get_courses(courses_text)
    calendar = icalendar.Calendar()
    calendar['version'] = "2.0"
    calendar['x-wr-timezone'] = 'Asia/Kolkata'
    rows = tuple(map(split_timetable_line, timetable_text.splitlines()))
    theory_slot_timings = get_slot_times(rows[0][2:], rows[1][1:])
    add_events((row[2:] for row in rows[4::2]), theory_slot_timings, courses, semester_dates, calendar)
    lab_slot_timings = get_slot_times(rows[2][2:], rows[3][1:])
    add_events((row[1:] for row in rows[5::2]), lab_slot_timings, courses, semester_dates, calendar)
    return calendar.to_ical()

In [ ]:
import icalendar
import datetime
from dateutil import rrule as dateutil_rrule
import re


days = ("MO", "TU", "WE", "TH", "FR", "SA", "SU")


RECORD_SIZE = 16
SHIFTS = {
    "REGISTRATION": 6,
    "CLASS_NUMBER": 7,
}


def transform_arrear_course(data: list[str], line_index: int, lines_removed: set[int]) -> int:
    if data[line_index + SHIFTS["REGISTRATION"]].strip() != "Reregistered":
        return 0
    lines_removed.add(line_index + SHIFTS['CLASS_NUMBER'])
    lines_removed.add(line_index + RECORD_SIZE)
    return 2


def transform_arrear_courses(data: list[str], start_index: int) -> list[str]:
    lines_removed = set()
    line_index = start_index
    while line_index < len(data):
        iterator_shift = transform_arrear_course(data, line_index, lines_removed)
        line_index += RECORD_SIZE + iterator_shift
    return [x for i, x in enumerate(data) if i not in lines_removed]

def get_courses(text: str) -> dict[str, dict]:
    """
    Converts the text copied from the course list in, VTopCC >> Academics >> Time Table, to a list of courses with the
    relevant data.
    """
    data = text.splitlines()
    data = [x for x in data if x.strip()]
    # print('\n'.join(data))
    start_index = data.index('1')
    data = transform_arrear_courses(data, start_index)
    courses = {}
    for line_index in range(start_index, len(data), RECORD_SIZE):
        slot = data[line_index + 7]
        header = (data[line_index + 2].split(' - '))
        course_code = header[0]
        courses[course_code] = {
            'title': header[1],
            'LTPJC': tuple(map(float, data[line_index + 4].split())),
            'class_code': data[line_index + 7],
            'slot': slot,
            'venue': data[line_index + 9],
            'professor': data[line_index + 10].rstrip(' -')
        }
    return courses


def get_slot_times(start_times: list[str], end_times: list[str]) -> list[(datetime.time, datetime.time)]:
    """
    Slots times from first two lines of timetable text becomes,
     list(tuple(start_time, end_time), ...) where,
     time = list(hours, minutes)
    """
    for times in (start_times, end_times):
        noon_flag = False
        # Bug - [12am, 1am) would be evaluated as pm
        previous_time = datetime.time(0)
        for index, time in enumerate(times):
            if time == "Lunch":
                continue
            if time == "-":
              continue
            time_list = list(int(component) for component in time.split(":"))
            time = datetime.time(*time_list)
            if time < previous_time:
                noon_flag = True
            if noon_flag:
                time_list[0] += 12
                time = datetime.time(*time_list)
            if not noon_flag:
                previous_time = time
            times[index] = time
    slot_times = list(zip(start_times, end_times))
    return slot_times



def add_events(
    day_rows: list[str],
    slot_timings: list[tuple[datetime.time, datetime.time]],
    courses: dict[str, dict],
    semester_dates: list[datetime.date],
    calendar: icalendar.cal.Calendar
) -> None:
    """Add class events to the calendar from timetable slots."""

    # Map day index to RRULE days
    days = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']

    for day_index, day_row in enumerate(day_rows):
        for slot_index, slot_cell in enumerate(day_row):
            if "-" not in slot_cell or slot_cell.strip() == "-":
                continue

            slot_cell_parts = slot_cell.split("-")
            if len(slot_cell_parts) < 4:
                continue  # Skip badly formatted cells

            slot_course = slot_cell_parts[1].strip()
            slot_venue = "-".join(slot_cell_parts[3:5]).strip()

            if slot_course not in courses:
                continue  # Skip unknown courses

            course = courses[slot_course]

            event = icalendar.Event()
            event.add('summary', course['title'])
            event.add('location', slot_venue)

            event.add('description', f"course_code = {slot_course}\n" +
                      "\n".join(f"{k} = {v}" for k, v in course.items()))

            semester_start = semester_dates[0]
            start_time, end_time = slot_timings[slot_index]

            dtstart = datetime.datetime.combine(semester_start, start_time)
            dtend = datetime.datetime.combine(semester_start, end_time)

            event.add('dtstart', dtstart)
            event.add('dtend', dtend)
            event.add('dtstamp', datetime.datetime.now())
            event.add('uid', f"{day_index}-{slot_index}-{datetime.datetime.now().timestamp()}")

            # Add RRULE for weekly repeat
            event.add('rrule', {'freq': 'weekly', 'byday': days[day_index]})

            # Add EXDATEs for semester breaks
            for i in range(1, len(semester_dates) - 1, 2):
                break_start = semester_dates[i]
                break_end = semester_dates[i + 1]

                exdates = list(dateutil_rrule.rrule(
                    dateutil_rrule.WEEKLY,
                    dtstart=datetime.datetime.combine(break_start, start_time),
                    until=datetime.datetime.combine(break_end, start_time),
                    byweekday=day_index
                ))

                for exdate in exdates:
                    event.add('exdate', exdate)

            calendar.add_component(event)



def split_timetable_line(line):
    return line.split('\t')


def split_text(page_text: str) -> tuple[str, str]:
    """
    Splits the text copied from VTopCC >> Academics >> Time Table to,
    courses_text, timetable_text
    """
    split_text = page_text.split("Total Number Of Credits:")
    courses_text = split_text[0]
    timetable_text = split_text[1][split_text[1].index("THEORY"):]
    return courses_text, timetable_text


def generate_calendar(
        page_text : str,
        semester_dates: list[datetime.date]
) -> bytes:
    """
    semester_dates: End date is exclusive
    """
    courses_text, timetable_text = split_text(page_text)
    courses = get_courses(courses_text)
    calendar = icalendar.Calendar()
    calendar['version'] = "2.0"
    calendar['x-wr-timezone'] = 'Asia/Kolkata'
    rows = tuple(map(split_timetable_line, timetable_text.splitlines()))
    theory_slot_timings = get_slot_times(rows[0][2:], rows[1][1:])
    add_events((row[2:] for row in rows[4::2]), theory_slot_timings, courses, semester_dates, calendar)
    lab_slot_timings = get_slot_times(rows[2][2:], rows[3][1:])
    add_events((row[1:] for row in rows[5::2]), lab_slot_timings, courses, semester_dates, calendar)
    return calendar.to_ical()

page_text = '''
Sl.No	Class Group	Course	L T P J C	Category	Course Option	Class Id	Slot/ Venue	Faculty Details	Registered / Updated Date & Time	Attendance Date/ Type	Status & Ref. No.
1

General (Semester)

ICSE103E - Computer Programming : Java

( Embedded Theory )

1 0 0 0 1.0

Foundation Core
Regular

VL2025260103939

TCC1 -


SJT118

BALA GANESH N -


SCORE

28-Jun-2025 19:01

29-Jun-2025
- Manual

Registered and Approved

2

General (Semester)

ICSE103E - Computer Programming : Java

( Embedded Lab )

0 0 4 0 2.0

Foundation Core
Regular

VL2025260103953

L49+L50+L57+L58 -


SJT216

BALA GANESH N -


SCORE

28-Jun-2025 19:01

29-Jun-2025
- Manual

Registered and Approved

3

General (Semester)

IENG102P - Technical Report Writing

( Lab Only )

0 0 2 0 1.0

Foundation Core
Regular

VL2025260100784

L53+L54 -


SJT519

RUKMINI S -


SSL

28-Jun-2025 19:02

29-Jun-2025
- Manual

Registered and Approved

4

General (Semester)

IMAT201L - Complex Variables and Linear Algebra

( Theory Only )

3 1 0 0 4.0

Foundation Core
Regular

VL2025260100927

A1+TA1+TAA1 -


SJT205

KALAIVANI K -


SAS

28-Jun-2025 19:02

29-Jun-2025
- Manual

Registered and Approved

5

General (Semester)

ISTS101P - Qualitative Skills Practice I

( Soft Skill )

0 0 3 0 1.5

Foundation Core
Regular

VL2025260100209

G1+TG1 -


SJT803

SIXPHRASE (APT) -


SSL

28-Jun-2025 19:00

29-Jun-2025
- Manual

Registered and Approved

6

General (Semester)

ISWE102L - Data Structures and Algorithms

( Theory Only )

3 0 0 0 3.0

Discipline Core
Regular

VL2025260102669

B1+TB1 -


SJT211A

RAGHAVAN R -


SCORE

28-Jun-2025 19:03

29-Jun-2025
- Manual

Registered and Approved

7

General (Semester)

ISWE102P - Data Structures and Algorithms Lab

( Lab Only )

0 0 2 0 1.0

Discipline Core
Regular

VL2025260102696

L55+L56 -


SJT216

RAGHAVAN R -


SCORE

28-Jun-2025 19:03

29-Jun-2025
- Manual

Registered and Approved

8

General (Semester)

ISWE204L - Operating Systems

( Theory Only )

3 0 0 0 3.0

Discipline Core
Regular

VL2025260102753

C1+TC1 -


SJT807

ARIVUSELVAN K -


SCORE

28-Jun-2025 19:04

29-Jun-2025
- Manual

Registered and Approved

9

General (Semester)

ISWE204P - Operating Systems Lab

( Lab Only )

0 0 2 0 1.0

Discipline Core
Regular

VL2025260102776

L45+L46 -


SJT120

ARIVUSELVAN K -


SCORE

28-Jun-2025 19:04

29-Jun-2025
- Manual

Registered and Approved

10

General (Semester)

ISWE301L - Computer Architecture and Organization

( Theory Only )

3 0 0 0 3.0

Discipline Core
Regular

VL2025260102642

D1+TD1 -


SJT215

ASHA M M -


SCORE

28-Jun-2025 19:05

29-Jun-2025
- Manual

Registered and Approved

11

General (Semester)

ISWE304L - Software Architecture

( Theory Only )

3 0 0 0 3.0

Discipline Core
Regular

VL2025260102790

E1+TE1 -


SJT803

ASHA N -


SCORE

28-Jun-2025 19:05

29-Jun-2025
- Manual

Registered and Approved

Total Number Of Credits: 23.5
Dropped Course Details

THEORY	Start	08:00	09:00	10:00	11:00	12:00	-	Lunch	14:00	15:00	16:00	17:00	18:00	18:51	19:01
End	08:50	09:50	10:50	11:50	12:50	-	Lunch	14:50	15:50	16:50	17:50	18:50	19:00	19:50
LAB	Start	08:00	08:51	09:51	10:41	11:40	12:31	Lunch	14:00	14:51	15:51	16:41	17:40	18:31	-
End	08:50	09:40	10:40	11:30	12:30	13:20	Lunch	14:50	15:40	16:40	17:30	18:30	19:20	-
MON	THEORY	A1-IMAT201L-TH-SJT205-ALL	F1	D1-ISWE301L-TH-SJT215-ALL	TB1-ISWE102L-TH-SJT211A-ALL	TG1-ISTS101P-SS-SJT803-ALL	-	Lunch	A2	F2	D2	TB2	TG2	-	V3
LAB	L1	L2	L3	L4	L5	L6	Lunch	L31	L32	L33	L34	L35	L36	-
TUE	THEORY	B1-ISWE102L-TH-SJT211A-ALL	G1-ISTS101P-SS-SJT803-ALL	E1-ISWE304L-TH-SJT803-ALL	TC1-ISWE204L-TH-SJT807-ALL	TAA1-IMAT201L-TH-SJT205-ALL	-	Lunch	B2	G2	E2	TC2	TAA2	-	V4
LAB	L7	L8	L9	L10	L11	L12	Lunch	L37	L38	L39	L40	L41	L42	-
WED	THEORY	C1-ISWE204L-TH-SJT807-ALL	A1-IMAT201L-TH-SJT205-ALL	F1	V1	V2	-	Lunch	C2	A2	F2	TD2	TBB2	-	V5
LAB	L13	L14	L15	L16	L17	L18	Lunch	L43	L44	L45-ISWE204P-LO-SJT120-ALL	L46-ISWE204P-LO-SJT120-ALL	L47	L48	-
THU	THEORY	D1-ISWE301L-TH-SJT215-ALL	B1-ISWE102L-TH-SJT211A-ALL	G1-ISTS101P-SS-SJT803-ALL	TE1-ISWE304L-TH-SJT803-ALL	TCC1-ICSE103E-ETH-SJT118-ALL	-	Lunch	D2	B2	G2	TE2	TCC2	-	V6
LAB	L19	L20	L21	L22	L23	L24	Lunch	L49-ICSE103E-ELA-SJT216-ALL	L50-ICSE103E-ELA-SJT216-ALL	L51	L52	L53-IENG102P-LO-SJT519-ALL	L54-IENG102P-LO-SJT519-ALL	-
FRI	THEORY	E1-ISWE304L-TH-SJT803-ALL	C1-ISWE204L-TH-SJT807-ALL	TA1-IMAT201L-TH-SJT205-ALL	TF1	TD1-ISWE301L-TH-SJT215-ALL	-	Lunch	E2	C2	TA2	TF2	TDD2	-	V7
LAB	L25	L26	L27	L28	L29	L30	Lunch	L55-ISWE102P-LO-SJT216-ALL	L56-ISWE102P-LO-SJT216-ALL	L57-ICSE103E-ELA-SJT216-ALL	L58-ICSE103E-ELA-SJT216-ALL	L59	L60	-
SAT	THEORY	V8	X11	X12	Y11	Y12	-	Lunch	X21	Z21	Y21	W21	W22	-	V9
LAB	L71	L72	L73	L74	L75	L76	Lunch	L77	L78	L79	L80	L81	L82	-
SUN	THEORY	V10	Y11	Y12	X11	X12	-	Lunch	Y21	Z21	X21	W21	W22	-	V11
LAB	L83	L84	L85	L86	L87	L88	Lunch	L89	L90	L91	L92	L93	L94	-

'''

start_date = datetime.date.fromisoformat("2025-07-09")
end_date = datetime.date.fromisoformat("2025-11-14")

ics_text = generate_calendar(page_text, [start_date, end_date])

file_path = input("Enter file path for output: ") + ".ics"
with open(file_path, "wb") as file:
    file.write(ics_text)

Enter file path for output: perfectperfect


In [ ]:
import icalendar
import datetime
from dateutil import rrule as dateutil_rrule
from typing import List, Tuple
import re

days = ("MO", "TU", "WE", "TH", "FR", "SA", "SU")

RECORD_SIZE = 16
SHIFTS = {
    "REGISTRATION": 6,
    "CLASS_NUMBER": 7,
}

def transform_arrear_course(data: List[str], line_index: int, lines_removed: set[int]) -> int:
    try:
        if line_index + SHIFTS["REGISTRATION"] < len(data) and data[line_index + SHIFTS["REGISTRATION"]].strip() != "Reregistered":
            return 0
        if line_index + SHIFTS['CLASS_NUMBER'] < len(data):
            lines_removed.add(line_index + SHIFTS['CLASS_NUMBER'])
        if line_index + RECORD_SIZE < len(data):
            lines_removed.add(line_index + RECORD_SIZE)
        return 2
    except IndexError:
        # Probably incomplete record; skip safely
        return 0

def transform_arrear_courses(data: List[str], start_index: int) -> List[str]:
    lines_removed = set()
    line_index = start_index
    while line_index < len(data):
        iterator_shift = transform_arrear_course(data, line_index, lines_removed)
        line_index += RECORD_SIZE + iterator_shift
    return [x for i, x in enumerate(data) if i not in lines_removed]

def get_courses(text: str) -> dict[str, dict]:
    data = text.splitlines()
    data = [x for x in data if x.strip()]
    try:
        start_index = data.index('1')
    except ValueError:
        raise Exception("Course list start not found. Check input format.")

    data = transform_arrear_courses(data, start_index)
    courses = {}
    for line_index in range(start_index, len(data), RECORD_SIZE):
        if line_index + 10 >= len(data):
            continue  # Incomplete record, skip

        slot = data[line_index + 7]
        header = (data[line_index + 2].split(' - '))
        course_code = header[0]
        courses[course_code] = {
            'title': header[1] if len(header) > 1 else "Unknown Title",
            'LTPJC': tuple(map(float, data[line_index + 4].split())),
            'class_code': data[line_index + 7],
            'slot': slot,
            'venue': data[line_index + 9],
            'professor': data[line_index + 10].rstrip(' -')
        }
    return courses

def get_slot_times(start_times: List[str], end_times: List[str]) -> List[Tuple[datetime.time, datetime.time]]:
    """
    Converts string start and end times into a list of (start_time, end_time) tuples.
    Handles AM/PM transitions gracefully like a polite time wizard.
    """
    for times in (start_times, end_times):
        noon_flag = False
        previous_hour = 0
        for index, time in enumerate(times):
            if time in ("Lunch", "-"):
                continue
            time_list = [int(component) for component in time.split(":")]
            hour = time_list[0]

            # Noon handling
            if hour < previous_hour:
                noon_flag = True
            if noon_flag and hour < 12:
                hour += 12

            previous_hour = hour
            times[index] = datetime.time(hour, time_list[1])
    return list(zip(start_times, end_times))

def add_events(
    day_rows: List[List[str]],
    slot_timings: List[Tuple[datetime.time, datetime.time]],
    courses: dict,
    semester_dates: List[datetime.date],
    calendar: icalendar.cal.Calendar
) -> None:
    """Adds class events to the calendar from timetable slots with error resilience and infinite optimism."""

    rrule_days = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']

    for day_index, day_row in enumerate(day_rows):
        for slot_index, slot_cell in enumerate(day_row):
            if "-" not in slot_cell or slot_cell.strip() == "-":
                continue

            slot_cell_parts = slot_cell.split("-")
            if len(slot_cell_parts) < 4:
                print(f"Skipped badly formatted cell: {slot_cell}")
                continue

            slot_course = slot_cell_parts[1].strip()
            slot_venue = "-".join(slot_cell_parts[3:5]).strip()

            if slot_course not in courses:
                print(f"Unknown course code in timetable: {slot_course}")
                continue

            course = courses[slot_course]

            event = icalendar.Event()
            event.add('summary', course['title'])
            event.add('location', slot_venue)
            event.add('description', f"course_code = {slot_course}\n" +
                      "\n".join(f"{k} = {v}" for k, v in course.items()))

            semester_start = semester_dates[0]
            start_time, end_time = slot_timings[slot_index]
            dtstart = datetime.datetime.combine(semester_start, start_time)
            dtend = datetime.datetime.combine(semester_start, end_time)

            event.add('dtstart', dtstart)
            event.add('dtend', dtend)
            event.add('dtstamp', datetime.datetime.now())
            event.add('uid', f"{day_index}-{slot_index}-{datetime.datetime.now().timestamp()}")

            event.add('rrule', {'freq': 'weekly', 'byday': rrule_days[day_index]})

            # Add EXDATEs for breaks
            for i in range(1, len(semester_dates) - 1, 2):
                break_start = semester_dates[i]
                break_end = semester_dates[i + 1]
                exdates = list(dateutil_rrule.rrule(
                    dateutil_rrule.WEEKLY,
                    dtstart=datetime.datetime.combine(break_start, start_time),
                    until=datetime.datetime.combine(break_end, start_time),
                    byweekday=day_index
                ))
                for exdate in exdates:
                    event.add('exdate', exdate)

            calendar.add_component(event)

def split_timetable_line(line: str) -> List[str]:
    return line.split('\t')

def split_text(page_text: str) -> Tuple[str, str]:
    split_text = page_text.split("Total Number Of Credits:")
    if len(split_text) < 2 or "THEORY" not in split_text[1]:
        raise Exception("Timetable section not found. Check input formatting.")
    courses_text = split_text[0]
    timetable_text = split_text[1][split_text[1].index("THEORY"):]
    return courses_text, timetable_text

def generate_calendar(page_text: str, semester_dates: List[datetime.date]) -> bytes:
    courses_text, timetable_text = split_text(page_text)
    courses = get_courses(courses_text)

    calendar = icalendar.Calendar()
    calendar['version'] = "2.0"
    calendar['x-wr-timezone'] = 'Asia/Kolkata'

    rows = [split_timetable_line(row) for row in timetable_text.splitlines()]

    theory_slot_timings = get_slot_times(rows[0][2:], rows[1][1:])
    add_events([row[2:] for row in rows[4::2]], theory_slot_timings, courses, semester_dates, calendar)

    lab_slot_timings = get_slot_times(rows[2][2:], rows[3][1:])
    add_events([row[1:] for row in rows[5::2]], lab_slot_timings, courses, semester_dates, calendar)

    return calendar.to_ical()

# Example usage remains same


page_text = '''
Sl.No	Class Group	Course	L T P J C	Category	Course Option	Class Id	Slot/ Venue	Faculty Details	Registered / Updated Date & Time	Attendance Date/ Type	Status & Ref. No.
1

General (Semester)

ICSE103E - Computer Programming : Java

( Embedded Theory )

1 0 0 0 1.0

Foundation Core
Regular

VL2025260103939

TCC1 -


SJT118

BALA GANESH N -


SCORE

28-Jun-2025 19:01

29-Jun-2025
- Manual

Registered and Approved

2

General (Semester)

ICSE103E - Computer Programming : Java

( Embedded Lab )

0 0 4 0 2.0

Foundation Core
Regular

VL2025260103953

L49+L50+L57+L58 -


SJT216

BALA GANESH N -


SCORE

28-Jun-2025 19:01

29-Jun-2025
- Manual

Registered and Approved

3

General (Semester)

IENG102P - Technical Report Writing

( Lab Only )

0 0 2 0 1.0

Foundation Core
Regular

VL2025260100784

L53+L54 -


SJT519

RUKMINI S -


SSL

28-Jun-2025 19:02

29-Jun-2025
- Manual

Registered and Approved

4

General (Semester)

IMAT201L - Complex Variables and Linear Algebra

( Theory Only )

3 1 0 0 4.0

Foundation Core
Regular

VL2025260100927

A1+TA1+TAA1 -


SJT205

KALAIVANI K -


SAS

28-Jun-2025 19:02

29-Jun-2025
- Manual

Registered and Approved

5

General (Semester)

ISTS101P - Qualitative Skills Practice I

( Soft Skill )

0 0 3 0 1.5

Foundation Core
Regular

VL2025260100209

G1+TG1 -


SJT803

SIXPHRASE (APT) -


SSL

28-Jun-2025 19:00

29-Jun-2025
- Manual

Registered and Approved

6

General (Semester)

ISWE102L - Data Structures and Algorithms

( Theory Only )

3 0 0 0 3.0

Discipline Core
Regular

VL2025260102669

B1+TB1 -


SJT211A

RAGHAVAN R -


SCORE

28-Jun-2025 19:03

29-Jun-2025
- Manual

Registered and Approved

7

General (Semester)

ISWE102P - Data Structures and Algorithms Lab

( Lab Only )

0 0 2 0 1.0

Discipline Core
Regular

VL2025260102696

L55+L56 -


SJT216

RAGHAVAN R -


SCORE

28-Jun-2025 19:03

29-Jun-2025
- Manual

Registered and Approved

8

General (Semester)

ISWE204L - Operating Systems

( Theory Only )

3 0 0 0 3.0

Discipline Core
Regular

VL2025260102753

C1+TC1 -


SJT807

ARIVUSELVAN K -


SCORE

28-Jun-2025 19:04

29-Jun-2025
- Manual

Registered and Approved

9

General (Semester)

ISWE204P - Operating Systems Lab

( Lab Only )

0 0 2 0 1.0

Discipline Core
Regular

VL2025260102776

L45+L46 -


SJT120

ARIVUSELVAN K -


SCORE

28-Jun-2025 19:04

29-Jun-2025
- Manual

Registered and Approved

10

General (Semester)

ISWE301L - Computer Architecture and Organization

( Theory Only )

3 0 0 0 3.0

Discipline Core
Regular

VL2025260102642

D1+TD1 -


SJT215

ASHA M M -


SCORE

28-Jun-2025 19:05

29-Jun-2025
- Manual

Registered and Approved

11

General (Semester)

ISWE304L - Software Architecture

( Theory Only )

3 0 0 0 3.0

Discipline Core
Regular

VL2025260102790

E1+TE1 -


SJT803

ASHA N -


SCORE

28-Jun-2025 19:05

29-Jun-2025
- Manual

Registered and Approved

Total Number Of Credits: 23.5
Dropped Course Details

THEORY	Start	08:00	09:00	10:00	11:00	12:00	-	Lunch	14:00	15:00	16:00	17:00	18:00	18:51	19:01
End	08:50	09:50	10:50	11:50	12:50	-	Lunch	14:50	15:50	16:50	17:50	18:50	19:00	19:50
LAB	Start	08:00	08:51	09:51	10:41	11:40	12:31	Lunch	14:00	14:51	15:51	16:41	17:40	18:31	-
End	08:50	09:40	10:40	11:30	12:30	13:20	Lunch	14:50	15:40	16:40	17:30	18:30	19:20	-
MON	THEORY	A1-IMAT201L-TH-SJT205-ALL	F1	D1-ISWE301L-TH-SJT215-ALL	TB1-ISWE102L-TH-SJT211A-ALL	TG1-ISTS101P-SS-SJT803-ALL	-	Lunch	A2	F2	D2	TB2	TG2	-	V3
LAB	L1	L2	L3	L4	L5	L6	Lunch	L31	L32	L33	L34	L35	L36	-
TUE	THEORY	B1-ISWE102L-TH-SJT211A-ALL	G1-ISTS101P-SS-SJT803-ALL	E1-ISWE304L-TH-SJT803-ALL	TC1-ISWE204L-TH-SJT807-ALL	TAA1-IMAT201L-TH-SJT205-ALL	-	Lunch	B2	G2	E2	TC2	TAA2	-	V4
LAB	L7	L8	L9	L10	L11	L12	Lunch	L37	L38	L39	L40	L41	L42	-
WED	THEORY	C1-ISWE204L-TH-SJT807-ALL	A1-IMAT201L-TH-SJT205-ALL	F1	V1	V2	-	Lunch	C2	A2	F2	TD2	TBB2	-	V5
LAB	L13	L14	L15	L16	L17	L18	Lunch	L43	L44	L45-ISWE204P-LO-SJT120-ALL	L46-ISWE204P-LO-SJT120-ALL	L47	L48	-
THU	THEORY	D1-ISWE301L-TH-SJT215-ALL	B1-ISWE102L-TH-SJT211A-ALL	G1-ISTS101P-SS-SJT803-ALL	TE1-ISWE304L-TH-SJT803-ALL	TCC1-ICSE103E-ETH-SJT118-ALL	-	Lunch	D2	B2	G2	TE2	TCC2	-	V6
LAB	L19	L20	L21	L22	L23	L24	Lunch	L49-ICSE103E-ELA-SJT216-ALL	L50-ICSE103E-ELA-SJT216-ALL	L51	L52	L53-IENG102P-LO-SJT519-ALL	L54-IENG102P-LO-SJT519-ALL	-
FRI	THEORY	E1-ISWE304L-TH-SJT803-ALL	C1-ISWE204L-TH-SJT807-ALL	TA1-IMAT201L-TH-SJT205-ALL	TF1	TD1-ISWE301L-TH-SJT215-ALL	-	Lunch	E2	C2	TA2	TF2	TDD2	-	V7
LAB	L25	L26	L27	L28	L29	L30	Lunch	L55-ISWE102P-LO-SJT216-ALL	L56-ISWE102P-LO-SJT216-ALL	L57-ICSE103E-ELA-SJT216-ALL	L58-ICSE103E-ELA-SJT216-ALL	L59	L60	-
SAT	THEORY	V8	X11	X12	Y11	Y12	-	Lunch	X21	Z21	Y21	W21	W22	-	V9
LAB	L71	L72	L73	L74	L75	L76	Lunch	L77	L78	L79	L80	L81	L82	-
SUN	THEORY	V10	Y11	Y12	X11	X12	-	Lunch	Y21	Z21	X21	W21	W22	-	V11
LAB	L83	L84	L85	L86	L87	L88	Lunch	L89	L90	L91	L92	L93	L94	-

'''

start_date = datetime.date.fromisoformat("2025-07-09")
end_date = datetime.date.fromisoformat("2025-11-14")

ics_text = generate_calendar(page_text, [start_date, end_date])

file_path = input("Enter file path for output: ") + ".ics"
with open(file_path, "wb") as file:
    file.write(ics_text)

Enter file path for output: check
